In [1]:
from marubatsu import NpBoolBoard, Marubatsu

mb = Marubatsu(boardclass=NpBoolBoard)
mb.cmove(0, 0)
mb.cmove(1, 0)
mb.cmove(0, 1)
mb.cmove(2, 0)
mb.cmove(1, 2)
mb.cmove(1, 1)
mb.cmove(0, 2)
print(mb)

winner o
oxx
ox.
Oo.



In [2]:
import numpy as np

na2d = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(na2d)
print(np.sum(na2d))

[[1 2 3]
 [4 5 6]
 [7 8 9]]
45


In [3]:
print(np.sum(na2d, axis=0))

[12 15 18]


In [4]:
nasum = np.zeros((3, ))
for i in range(3):
    nasum[i] = np.sum(na2d[:, i])
print(nasum)

[12. 15. 18.]


In [5]:
print(np.sum(na2d, axis=1))

[ 6 15 24]


In [6]:
na3d = np.array([[[1, 2, 3], [4, 5, 6], [7, 8, 9]],
                 [[10, 11, 12], [13, 14, 15], [16, 17, 18]]])
print(na3d)
print(np.sum(na3d, axis=2))

[[[ 1  2  3]
  [ 4  5  6]
  [ 7  8  9]]

 [[10 11 12]
  [13 14 15]
  [16 17 18]]]
[[ 6 15 24]
 [33 42 51]]


In [7]:
nasum = np.zeros((2, 3))
for i in range(2):
    for j in range(3):
        nasum[i][j] = np.sum(na3d[i, j, :])
print(nasum)

[[ 6. 15. 24.]
 [33. 42. 51.]]


In [8]:
print(np.sum(na2d, axis=(0, 1)))

45


In [9]:
print(np.sum(na3d, axis=(0, 1)))

[51 57 63]


In [10]:
nasum = np.zeros((3, ))
for i in range(3):
    nasum[i] = np.sum(na3d[:, :, i])
print(nasum)

[51. 57. 63.]


In [11]:
print(mb)
print(np.count_nonzero(mb.board.board, axis=1))

winner o
oxx
ox.
Oo.

[[1 1 2]
 [2 1 0]]


In [12]:
last_turn = mb.board.CIRCLE
turn = mb.board.CROSS
rowdata = np.count_nonzero(mb.board.board, axis=1)
for y in range(mb.board.BOARD_SIZE):
    turn_count = rowdata[turn, y]
    lastturn_count = rowdata[last_turn, y]
    emptycount = mb.board.BOARD_SIZE - turn_count - lastturn_count
    print((lastturn_count, turn_count, emptycount))

(np.int64(1), np.int64(2), np.int64(0))
(np.int64(1), np.int64(1), np.int64(1))
(np.int64(2), np.int64(0), np.int64(1))


In [13]:
# 対角線 1（左上から右下方向）のマークの計算
print(np.diagonal(mb.board.board, axis1=1, axis2=2))
# 対角線 2（左下から右上方向）のマークの計算
print(np.diagonal(np.flip(mb.board.board, axis=2), axis1=1, axis2=2))

[[ True False False]
 [False  True False]]
[[ True False False]
 [False  True  True]]


In [14]:
# 対角線 1 のマークの数の計算
print(np.count_nonzero(np.diagonal(mb.board.board, axis1=1, axis2=2), axis=1))
# 対角線 2 のマークの数の計算
print(np.count_nonzero(np.diagonal(np.flip(mb.board.board, axis=2), 
                                   axis1=1, axis2=2), axis=1))

[1 1]
[1 2]


In [15]:
from marubatsu import NpBoolBoard
from collections import defaultdict

def count_markpats2(self, turn, last_turn):
    markpats = defaultdict(int)
    
    if self.count_linemark:
        for countdict in [self.rowcount, self.colcount, self.diacount]:
            for circlecount, crosscount in zip(countdict[self.CIRCLE], countdict[self.CROSS]):
                emptycount = self.BOARD_SIZE - circlecount - crosscount
                if last_turn == self.CIRCLE:
                    markpats[(circlecount, crosscount, emptycount)] += 1
                else:
                    markpats[(crosscount, circlecount, emptycount)] += 1
    else:
        # 行と列のマークのパターンの計算
        for axis in [1, 2]:
            data = np.count_nonzero(mb.board.board, axis=axis)
            for i in range(mb.board.BOARD_SIZE):
                turn_count = data[turn, i]
                lastturn_count = data[last_turn, i]
                emptycount = mb.board.BOARD_SIZE - turn_count - lastturn_count
                markpats[(lastturn_count, turn_count, emptycount)] += 1

        # 対角線のマークのパターンの計算          
        diagdata = [
            np.count_nonzero(np.diagonal(mb.board.board, axis1=1, axis2=2), axis=1),
            np.count_nonzero(np.diagonal(np.flip(mb.board.board, axis=2),
                                         axis1=1, axis2=2), axis=1),
        ]
        for data in diagdata:
            turn_count = data[turn]
            lastturn_count = data[last_turn]
            emptycount = mb.board.BOARD_SIZE - turn_count - lastturn_count
            markpats[(lastturn_count, turn_count, emptycount)] += 1

    return markpats

NpBoolBoard.count_markpats2 = count_markpats2

In [16]:
from pprint import pprint
pprint(mb.board.count_markpats2(mb.turn, mb.last_turn))

defaultdict(<class 'int'>,
            {(np.int64(0), np.int64(1), np.int64(2)): 1,
             (np.int64(1), np.int64(1), np.int64(1)): 2,
             (np.int64(1), np.int64(2), np.int64(0)): 3,
             (np.int64(2), np.int64(0), np.int64(1)): 1,
             (np.int64(3), np.int64(0), np.int64(0)): 1})


In [17]:
%timeit mb.count_markpats()
%timeit mb.board.count_markpats2(mb.turn, mb.last_turn)

18.6 μs ± 939 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
30.9 μs ± 1.67 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [18]:
mb.restart()
print(mb.board.count_markpats2(mb.turn, mb.last_turn))

TypeError: unhashable type: 'numpy.ndarray'

In [19]:
turn = mb.turn
last_turn = mb.last_turn
data = np.count_nonzero(mb.board.board, axis=0)
for i in range(mb.board.BOARD_SIZE):
    turn_count = data[turn, i]
    lastturn_count = data[last_turn, i]
    print(turn_count)
    print(lastturn_count)

0
[[0 0 0]]
0
[[0 0 0]]
0
[[0 0 0]]


In [20]:
print(last_turn)

None


In [21]:
na = np.array([[1, 2, 3], [4, 5, 6]])
print(na)
print(na.shape)
na2 = na[None]
print(na2)
print(na2.shape)

[[1 2 3]
 [4 5 6]]
(2, 3)
[[[1 2 3]
  [4 5 6]]]
(1, 2, 3)


In [22]:
def restart(self):
    self.initialize_board()
    self.turn = self.CIRCLE     
    self.move_count = 0
    self.status = self.PLAYING
    self.last_move = -1, -1          
    self.last_turn = self.CROSS
    self.records = [self.last_move]
    
Marubatsu.restart = restart

In [23]:
mb.restart()
print(mb.board.count_markpats2(mb.turn, mb.last_turn))

defaultdict(<class 'int'>, {(np.int64(0), np.int64(0), np.int64(3)): 8})


In [24]:
for board_size in [10, 50, 100]:
    mb = Marubatsu(boardclass=NpBoolBoard, board_size=board_size)
    print(f"board_size = {board_size}")
    print("count_markpats")
    %timeit mb.count_markpats()
    print("count_markpats2")
    %timeit mb.board.count_markpats2(mb.turn, mb.last_turn)
    print()

board_size = 10
count_markpats
42.4 μs ± 1.65 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
count_markpats2
39.2 μs ± 1.47 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

board_size = 50
count_markpats
171 μs ± 2.27 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
count_markpats2
95.4 μs ± 1.48 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

board_size = 100
count_markpats
349 μs ± 17.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
count_markpats2
180 μs ± 1.48 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

